In [1]:
"""
    Training pipeline for food model 
"""
import time
from datetime import datetime
from pprint import pprint
import pandas as pd

from airflow import DAG
from airflow.operators.python import PythonOperator, PythonVirtualenvOperator

from dags.utils.config import VERSION
from dags.utils.transformers import *
from dags.pipelines.model_pipeline.data import collect_milk_data, collect_prep_data, collect_bank_data
from dags.pipelines.model_pipeline.data import merge_data, preprocess_data

import pandas as pd
import numpy as np
import locale
import sys
import sklearn
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
AIRFLOW_HOME = os.getenv('AIRFLOW_HOME')

base_path = os.path.join(AIRFLOW_HOME, 'dags/pipelines/model_pipeline/')

collect_milk_data(base_path, 'precio_leche.csv')

collect_bank_data(base_path, 'banco_central.csv' )

collect_prep_data(base_path, 'precipitaciones.csv')

merge_data(base_path)

preprocess_data(base_path)


Error: unsupported locale setting

In [87]:
df_merge.head()

,anio,mes,Coquimbo,Valparaiso,Metropolitana_de_Santiago,Libertador_Gral__Bernardo_O_Higgins,Maule,Biobio,La_Araucania,Los_Rios,PIB_Agropecuario_silvicola,PIB_Pesca,PIB_Mineria,PIB_Mineria_del_cobre,PIB_Otras_actividades_mineras,PIB_Industria_Manufacturera,PIB_Alimentos,PIB_Bebidas_y_tabaco,PIB_Textil,PIB_Maderas_y_muebles,PIB_Celulosa,PIB_Refinacion_de_petroleo,PIB_Quimica,PIB_Minerales_no_metalicos_y_metalica_basica,PIB_Productos_metalicos,PIB_Electricidad,PIB_Construccion,PIB_Comercio,PIB_Restaurantes_y_hoteles,PIB_Transporte,PIB_Comunicaciones,PIB_Servicios_financieros,PIB_Servicios_empresariales,PIB_Servicios_de_vivienda,PIB_Servicios_personales,PIB_Administracion_publica,PIB_a_costo_de_factores,PIB,Imacec_empalmado,Imacec_produccion_de_bienes,Imacec_minero,Imacec_industria,Imacec_resto_de_bienes,Imacec_comercio,Imacec_servicios,Imacec_a_costo_de_factores,Imacec_no_minero,Indice_de_ventas_comercio_real_no_durables_IVCM,Precio_leche
Periodo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014.0-1.0,2014.0,1.0,3.524673,2.378766,7.278272,4.804760,7.831522,33.684003,96.707966,130.212946,55.059.326,449.097.098,123.408.811,110.892.244,125.165.676,125.915.256,323.829.181,160.131.672,271.608.802,613.149.007,111.186.684,107.176.547,197.004.194,6.795.359,203.394.915,309.290.133,635.812.268,10.768.256,220.170.996,539.108.727,364.948.817,603.426.797,126.672.311,85.785.202,787.099.521,522.480.077,102.724.817,112.152.352,976.113.405,101.540.937,977.900.376,985.915.778,107.470.632,103.244.622,93.851.301,97.687.997,975.892.915,943.909.236,219.03
2014.0-2.0,2014.0,2.0,1.688856,8.389854,17.803070,7.295178,3.015932,22.186197,43.157411,65.228010,589.162.366,530.628.153,118.979.099,107.686.125,112.929.737,117.324.454,326.848.287,151.002.582,24.777.101,58.489.291,985.263.519,987.746.023,169.050.011,637.721.806,182.004.135,276.725.914,621.419.201,106.567.859,21.519.997,530.525.285,329.424.874,602.969.763,116.462.931,829.465.384,690.060.504,523.184.092,985.454.361,107.497.828,935.602.943,982.574.646,942.799.014,918.649.843,107.454.234,102.175.861,888.267.283,9.371.354,934.715.034,928.126.259,227.45
2014.0-3.0,2014.0,3.0,2.237203,5.082094,5.403087,15.025209,31.280017,78.854388,128.113100,172.223450,53.363.658,107.007.804,127.819.883,115.400.625,124.192.583,134.314.265,397.250.136,156.581.527,291.436.774,663.043.838,102.251.828,115.536.937,197.408.229,757.357.597,202.930.175,290.911.613,710.059.845,122.425.043,211.265.323,561.815.961,344.621.056,595.793.932,117.932.685,825.405.999,142.600.046,525.208.459,111.566.458,121.659.577,105.885.915,107.308.181,101.285.403,105.167.997,114.516.992,11.737.952,103.080.921,106.096.113,106.453.564,101.583.412,232.47
2014.0-4.0,2014.0,4.0,0.954797,2.044854,11.612500,17.549566,49.346997,91.352068,108.177327,147.356305,325.108.389,935.169.145,123.033.382,110.466.031,125.673.509,132.699.445,371.727.747,167.407.156,312.688.204,675.912.639,984.276.626,100.932.989,200.687.899,735.710.672,215.379.842,282.374.957,742.671.945,111.123.103,218.150.367,548.235.516,354.969.936,587.095.158,124.245.508,829.519.906,147.877.931,523.399.237,10.894.836,118.497.051,103.133.424,10.071.414,974.925.445,103.903.594,100.708.701,106.543.369,105.138.498,103.606.386,103.829.441,963.546.529,244.29
2014.0-5.0,2014.0,5.0,2.484941,22.233003,59.984144,134.375287,174.540865,196.509109,250.971478,353.532545,267.626.474,754.664.357,134.515.937,12.154.244,129.734.967,131.618.094,366.221.406,166.446.238,311.158.306,700.194.177,104.642.347,108.895.145,190.204.766,73.216.809,205.418.979,301.020.742,727.233.884,107.559.583,211.949.063,54.856.123,360.450.365,59.077.053,124.993.379,831.266.876,150.703.515,52.391.317,109.321.638,119.082.935,103.643.345,101.511.782,106.591.405,103.056.896,956.634.254,103.126.713,105.888.966,103.961.362,103.279.589,998.625.376,243.44


In [82]:
pd.options.display.max_columns = 200
merge[[col for col in merge.columns if 'std' in col]].head()

,Coquimbo_rolling3_std,Valparaiso_rolling3_std,Metropolitana_de_Santiago_rolling3_std,Libertador_Gral__Bernardo_O_Higgins_rolling3_std,Maule_rolling3_std,Biobio_rolling3_std,La_Araucania_rolling3_std,Los_Rios_rolling3_std,PIB_Agropecuario_silvicola_rolling3_std,PIB_Pesca_rolling3_std,PIB_Mineria_rolling3_std,PIB_Mineria_del_cobre_rolling3_std,PIB_Otras_actividades_mineras_rolling3_std,PIB_Industria_Manufacturera_rolling3_std,PIB_Alimentos_rolling3_std,PIB_Bebidas_y_tabaco_rolling3_std,PIB_Textil_rolling3_std,PIB_Maderas_y_muebles_rolling3_std,PIB_Celulosa_rolling3_std,PIB_Refinacion_de_petroleo_rolling3_std,PIB_Quimica_rolling3_std,PIB_Minerales_no_metalicos_y_metalica_basica_rolling3_std,PIB_Productos_metalicos_rolling3_std,PIB_Electricidad_rolling3_std,PIB_Construccion_rolling3_std,PIB_Comercio_rolling3_std,PIB_Restaurantes_y_hoteles_rolling3_std,PIB_Transporte_rolling3_std,PIB_Comunicaciones_rolling3_std,PIB_Servicios_financieros_rolling3_std,PIB_Servicios_empresariales_rolling3_std,PIB_Servicios_de_vivienda_rolling3_std,PIB_Servicios_personales_rolling3_std,PIB_Administracion_publica_rolling3_std,PIB_a_costo_de_factores_rolling3_std,PIB_rolling3_std,Imacec_empalmado_rolling3_std,Imacec_produccion_de_bienes_rolling3_std,Imacec_minero_rolling3_std,Imacec_industria_rolling3_std,Imacec_resto_de_bienes_rolling3_std,Imacec_comercio_rolling3_std,Imacec_servicios_rolling3_std,Imacec_a_costo_de_factores_rolling3_std,Imacec_no_minero_rolling3_std,Indice_de_ventas_comercio_real_no_durables_IVCM_rolling3_std
0,1.298119,4.250481,7.442156,1.760992,3.405137,8.130176,37.865961,45.951288,3.776679e+08,5.765116e+07,3.132279e+06,2.267068e+06,8.652115e+06,6.074614e+06,2.134830e+06,6.455241e+06,1.745364e+08,3.922036e+08,6.180657e+08,6.226566e+08,1.976659e+07,4.461324e+08,1.512557e+07,2.302638e+07,1.017744e+07,6.774055e+07,1.404675e+08,6.069410e+06,2.511922e+07,3.231718e+05,7.219122e+06,5.258613e+08,6.861695e+07,4.978138e+05,6.241840e+08,3.291245e+06,2.899138,2.361737,2.474874,4.737615,0.011314,0.755897,3.571596,2.812164,2.899138,1.060660
1,0.942381,3.010604,6.683902,5.329371,15.121069,29.955098,42.956205,53.907248,3.088552e+08,2.247690e+08,4.420395e+06,3.875526e+06,6.800941e+06,8.495086e+06,4.154550e+07,4.601964e+06,1.485634e+08,3.355650e+08,5.072474e+08,5.060009e+08,1.625725e+07,4.032626e+08,1.221802e+07,1.632703e+07,4.756929e+07,6.041006e+07,1.122087e+08,1.616787e+07,1.782364e+07,4.281006e+06,5.519249e+06,4.281969e+08,3.474932e+08,1.416437e+06,5.071110e+08,7.218114e+06,6.309245,4.607622,3.542585,6.683507,4.072637,8.481103,7.241149,6.323775,6.675166,4.698106
2,0.643440,3.173461,6.199994,5.342880,23.351809,36.858725,44.426894,55.993237,2.679086e+08,4.141173e+08,4.425443e+06,3.907079e+06,6.969560e+06,9.377737e+06,3.564169e+07,8.340952e+06,1.604430e+08,3.528133e+08,5.095224e+08,5.078384e+08,1.739684e+07,6.374837e+07,1.686629e+07,7.141665e+06,6.274677e+07,8.164296e+06,1.115902e+08,1.569069e+07,1.284895e+07,7.949469e+06,4.134820e+06,2.359584e+06,3.145639e+08,1.111878e+06,5.359697e+08,7.433478e+06,6.503287,4.704419,3.548015,7.379668,6.904603,7.822951,8.898352,6.551640,6.904470,4.421560
3,0.821308,10.885310,29.881612,68.189763,78.020928,64.623046,77.332384,112.546290,1.432119e+08,4.354867e+08,5.767680e+06,5.823712e+07,2.869570e+06,1.356853e+06,1.655543e+07,5.992080e+06,1.185259e+07,1.886508e+07,5.085486e+08,7.311915e+06,5.362558e+06,3.888906e+08,6.587958e+06,9.333937e+06,1.631375e+07,7.761169e+06,3.793141e+06,2.888529e+08,8.038464e+06,3.073933e+08,3.878669e+06,3.009042e+06,4.113113e+06,2.724603e+08,5.748572e+07,1.682454e+06,1.464022,3.600473,4.613772,1.062342,9.785768,7.436300,1.453887,1.346870,1.696180,2.687600
4,28.317255,55.046609,52.672798,86.510624,112.290861,135.467678,129.716413,141.886443,5.390006e+07,1.101323e+08,5.805963e+06,5.878257e+07,2.899863e+06,5.049206e+06,1.919717e+07,5.914778e+06,1.642337e+08,3.592128e+08,5.089242e+08,4.891492e+08,6.537191e+06,3.640158e+08,9.340147e+06,9.532484e+06,2.298419e+07,5.744788e+07,3.338413e+06,2.818418e+0

In [80]:
merge.shape

(74, 141)

In [103]:
# Features
TARGET_COL = 'Precio_leche'

PERIOD_COLS = [
    'anio',
    'mes',
]

PIB_COLS = [
    'PIB_Agropecuario_silvicola',
    'PIB_Pesca',
    'PIB_Mineria',
    'PIB_Mineria_del_cobre',
    'PIB_Otras_actividades_mineras',
    'PIB_Industria_Manufacturera',
    'PIB_Alimentos',
    'PIB_Bebidas_y_tabaco',
    'PIB_Textil',
    'PIB_Maderas_y_muebles',
    'PIB_Celulosa',
    'PIB_Refinacion_de_petroleo',
    'PIB_Quimica',
    'PIB_Minerales_no_metalicos_y_metalica_basica',
    'PIB_Productos_metalicos',
    'PIB_Electricidad',
    'PIB_Construccion',
    'PIB_Comercio',
    'PIB_Restaurantes_y_hoteles',
    'PIB_Transporte',
    'PIB_Comunicaciones',
    'PIB_Servicios_financieros',
    'PIB_Servicios_empresariales',
    'PIB_Servicios_de_vivienda',
    'PIB_Servicios_personales',
    'PIB_Administracion_publica',
    'PIB_a_costo_de_factores',
    'PIB',
]

IMACEC_INDICE_COLS = [
    'Imacec_empalmado',
    'Imacec_produccion_de_bienes',
    'Imacec_minero',
    'Imacec_industria',
    'Imacec_resto_de_bienes',
    'Imacec_comercio',
    'Imacec_servicios',
    'Imacec_a_costo_de_factores',
    'Imacec_no_minero',
    'Indice_de_ventas_comercio_real_no_durables_IVCM'
]

CITY_COLS = [
    'Coquimbo',
    'Valparaiso',
    'Metropolitana_de_Santiago',
    'Libertador_Gral__Bernardo_O_Higgins',
    'Maule',
    'Biobio',
    'La_Araucania',
    'Los_Rios'
]

MILK_COLS = PERIOD_COLS

BANK_COLS = PERIOD_COLS + PIB_COLS + IMACEC_INDICE_COLS

PREP_COLS = PERIOD_COLS + CITY_COLS

MERGE_COLS = PERIOD_COLS + CITY_COLS + PIB_COLS + IMACEC_INDICE_COLS 



In [159]:
"""
    This file contains transformer classes for scikit learn pipelines.
    The intention is to reuse the code
"""
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
    
class TakeVariables(BaseEstimator, TransformerMixin):
    """
        Take a list of existing variables from the dataframe
    """
    def __init__(self, cols: list):
        assert len(cols) > 0
        self.columns = cols
    
    def fit(self, X: pd.DataFrame, y=None):
        return self
    
    def transform(self, X: pd.DataFrame):
        X = X.copy()
        valid_cols = [col for col in self.columns if col in X.columns]
        X = X[valid_cols]
        return X

class DropNaTransformer(BaseEstimator, TransformerMixin):
    """
        Take a list of existing variables from the dataframe
    """
    def __init__(self, cols: list=[]):
        self.cols = cols
    
    def fit(self, X: pd.DataFrame, y=None):
        return self

    def fit_transform(self, X: pd.DataFrame, y=None):
        mask = (np.isnan(X).any(1) | np.isnan(y))
        if hasattr(X, 'loc'):
            X = X.loc[~mask]
        else:
            X = X[~mask]
        if hasattr(y, 'loc'):
            y = y.loc[~mask]
        else:
            y = y[~mask]
        
        return X, y  
    
    def transform(self, X: pd.DataFrame):
        mask = np.isnan(X).any(-1)
        if hasattr(X, 'loc'):
            X = X.loc[~mask]
        else:
            X = X[~mask]
        print('holaaaa')
        return X

class FixingFormattedString(BaseEstimator, TransformerMixin):
    """
        Fix financial numbers 
    """
    def __init__(self, cols: list, cols_type: str):
        self.cols = cols
        self.cols_type = cols_type

    def fit(self, X: pd.DataFrame, y=None):
        return self

    def casting_finance(self, x):
        if self.cols_type == 'PIB':
            return int(x.replace('.', ''))
        else: 
            x = x.split('.')
            if x[0].startswith('1'): #es 100+
                if len(x[0]) >2:
                    return float(x[0] + '.' + x[1])
                else:
                    x = x[0]+x[1]
                    return float(x[0:3] + '.' + x[3:])
            else:
                if len(x[0])>2:
                    return float(x[0][0:2] + '.' + x[0][-1])
                else:
                    x = x[0] + x[1]
                    return float(x[0:2] + '.' + x[2:])

    def transform(self, X: pd.DataFrame):
        X = X.copy()
        for col in self.cols:
            if col in X.columns and (X[col].dtypes == 'str' or X[col].dtypes == 'object'):
                X[col] = X[col].apply(lambda x: self.casting_finance(x))
        return X

class RollingTransformer(BaseEstimator, TransformerMixin):
    """
        Perform rolling or shift operations over a set of existing variables
    """
    def __init__(self, cols: list, method: str='mean', window_size: int=3):
        self.cols = cols
        self.window_size = window_size
        self.method = method

    def fit(self, X: pd.DataFrame, y=None):
        return self

    def transform(self, X: pd.DataFrame):
        X = X.copy()

        feats = []
        for col in self.cols:
            if col in X.columns:
                if self.method == 'mean':
                    feat = pd.DataFrame(X[col].rolling(window=self.window_size, min_periods=1).mean())
                elif self.method == 'std':
                    feat = pd.DataFrame(X[col].rolling(window=self.window_size, min_periods=1).std())
                else:
                    raise NotImplementedError
                feat.columns = [f'{col}_rolling{self.window_size}_{self.method}']
                feats.append(feat)

        if len(feats):
            feats = pd.concat(feats, axis=1)
            X = pd.concat([X, feats], axis=1)
            
        return X

In [101]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, mutual_info_regression
import os

In [123]:
# Features
TARGET_COL = 'Precio_leche'

PERIOD_COLS = [
    'anio',
    'mes',
]

PIB_COLS = [
    'PIB_Agropecuario_silvicola',
    'PIB_Pesca',
    'PIB_Mineria',
    'PIB_Mineria_del_cobre',
    'PIB_Otras_actividades_mineras',
    'PIB_Industria_Manufacturera',
    'PIB_Alimentos',
    'PIB_Bebidas_y_tabaco',
    'PIB_Textil',
    'PIB_Maderas_y_muebles',
    'PIB_Celulosa',
    'PIB_Refinacion_de_petroleo',
    'PIB_Quimica',
    'PIB_Minerales_no_metalicos_y_metalica_basica',
    'PIB_Productos_metalicos',
    'PIB_Electricidad',
    'PIB_Construccion',
    'PIB_Comercio',
    'PIB_Restaurantes_y_hoteles',
    'PIB_Transporte',
    'PIB_Comunicaciones',
    'PIB_Servicios_financieros',
    'PIB_Servicios_empresariales',
    'PIB_Servicios_de_vivienda',
    'PIB_Servicios_personales',
    'PIB_Administracion_publica',
    'PIB_a_costo_de_factores',
    'PIB',
]

IMACEC_INDICE_COLS = [
    'Imacec_empalmado',
    'Imacec_produccion_de_bienes',
    'Imacec_minero',
    'Imacec_industria',
    'Imacec_resto_de_bienes',
    'Imacec_comercio',
    'Imacec_servicios',
    'Imacec_a_costo_de_factores',
    'Imacec_no_minero',
    'Indice_de_ventas_comercio_real_no_durables_IVCM'
]

CITY_COLS = [
    'Coquimbo',
    'Valparaiso',
    'Metropolitana_de_Santiago',
    'Libertador_Gral__Bernardo_O_Higgins',
    'Maule',
    'Biobio',
    'La_Araucania',
    'Los_Rios'
]

MILK_COLS = PERIOD_COLS

BANK_COLS = PERIOD_COLS + PIB_COLS + IMACEC_INDICE_COLS

PREP_COLS = PERIOD_COLS + CITY_COLS

MERGE_COLS = PERIOD_COLS + CITY_COLS + PIB_COLS + IMACEC_INDICE_COLS 

TAKE_VARS = [
    'Precio_leche_rolling3_mean',
    'Coquimbo_rolling3_mean',
    'Valparaiso_rolling3_mean',
    'Metropolitana_de_Santiago_rolling3_mean',
    'Libertador_Gral__Bernardo_O_Higgins_rolling3_mean',
    'Maule_rolling3_mean',
    'Biobio_rolling3_mean',
    'La_Araucania_rolling3_mean',
    'Los_Rios_rolling3_mean',
    'PIB_Agropecuario_silvicola_rolling3_mean',
    'PIB_Pesca_rolling3_mean',
    'PIB_Mineria_rolling3_mean',
    'PIB_Mineria_del_cobre_rolling3_mean',
    'PIB_Otras_actividades_mineras_rolling3_mean',
    'PIB_Industria_Manufacturera_rolling3_mean',
    'PIB_Alimentos_rolling3_mean',
    'PIB_Bebidas_y_tabaco_rolling3_mean',
    'PIB_Textil_rolling3_mean',
    'PIB_Maderas_y_muebles_rolling3_mean',
    'PIB_Celulosa_rolling3_mean',
    'PIB_Refinacion_de_petroleo_rolling3_mean',
    'PIB_Quimica_rolling3_mean',
    'PIB_Minerales_no_metalicos_y_metalica_basica_rolling3_mean',
    'PIB_Productos_metalicos_rolling3_mean',
    'PIB_Electricidad_rolling3_mean',
    'PIB_Construccion_rolling3_mean',
    'PIB_Comercio_rolling3_mean',
    'PIB_Restaurantes_y_hoteles_rolling3_mean',
    'PIB_Transporte_rolling3_mean',
    'PIB_Comunicaciones_rolling3_mean',
    'PIB_Servicios_financieros_rolling3_mean',
    'PIB_Servicios_empresariales_rolling3_mean',
    'PIB_Servicios_de_vivienda_rolling3_mean',
    'PIB_Servicios_personales_rolling3_mean',
    'PIB_Administracion_publica_rolling3_mean',
    'PIB_a_costo_de_factores_rolling3_mean',
    'PIB_rolling3_mean',
    'Imacec_empalmado_rolling3_mean',
    'Imacec_produccion_de_bienes_rolling3_mean',
    'Imacec_minero_rolling3_mean',
    'Imacec_industria_rolling3_mean',
    'Imacec_resto_de_bienes_rolling3_mean',
    'Imacec_comercio_rolling3_mean',
    'Imacec_servicios_rolling3_mean',
    'Imacec_a_costo_de_factores_rolling3_mean',
    'Imacec_no_minero_rolling3_mean',
    'Indice_de_ventas_comercio_real_no_durables_IVCM_rolling3_mean',
    'Precio_leche_rolling3_std',
    'Coquimbo_rolling3_std',
    'Valparaiso_rolling3_std',
    'Metropolitana_de_Santiago_rolling3_std',
    'Libertador_Gral__Bernardo_O_Higgins_rolling3_std',
    'Maule_rolling3_std',
    'Biobio_rolling3_std',
    'La_Araucania_rolling3_std',
    'Los_Rios_rolling3_std',
    'PIB_Agropecuario_silvicola_rolling3_std',
    'PIB_Pesca_rolling3_std',
    'PIB_Mineria_rolling3_std',
    'PIB_Mineria_del_cobre_rolling3_std',
    'PIB_Otras_actividades_mineras_rolling3_std',
    'PIB_Industria_Manufacturera_rolling3_std',
    'PIB_Alimentos_rolling3_std',
    'PIB_Bebidas_y_tabaco_rolling3_std',
    'PIB_Textil_rolling3_std',
    'PIB_Maderas_y_muebles_rolling3_std',
    'PIB_Celulosa_rolling3_std',
    'PIB_Refinacion_de_petroleo_rolling3_std',
    'PIB_Quimica_rolling3_std',
    'PIB_Minerales_no_metalicos_y_metalica_basica_rolling3_std',
    'PIB_Productos_metalicos_rolling3_std',
    'PIB_Electricidad_rolling3_std',
    'PIB_Construccion_rolling3_std',
    'PIB_Comercio_rolling3_std',
    'PIB_Restaurantes_y_hoteles_rolling3_std',
    'PIB_Transporte_rolling3_std',
    'PIB_Comunicaciones_rolling3_std',
    'PIB_Servicios_financieros_rolling3_std',
    'PIB_Servicios_empresariales_rolling3_std',
    'PIB_Servicios_de_vivienda_rolling3_std',
    'PIB_Servicios_personales_rolling3_std',
    'PIB_Administracion_publica_rolling3_std',
    'PIB_a_costo_de_factores_rolling3_std',
    'PIB_rolling3_std',
    'Imacec_empalmado_rolling3_std',
    'Imacec_produccion_de_bienes_rolling3_std',
    'Imacec_minero_rolling3_std',
    'Imacec_industria_rolling3_std',
    'Imacec_resto_de_bienes_rolling3_std',
    'Imacec_comercio_rolling3_std',
    'Imacec_servicios_rolling3_std',
    'Imacec_a_costo_de_factores_rolling3_std',
    'Imacec_no_minero_rolling3_std',
    'Indice_de_ventas_comercio_real_no_durables_IVCM_rolling3_std',
    'Precio_leche',
    'Coquimbo',
    'Valparaiso',
    'Metropolitana_de_Santiago',
    'Libertador_Gral__Bernardo_O_Higgins',
    'Maule',
    'Biobio',
    'La_Araucania',
    'Los_Rios',
    'PIB_Agropecuario_silvicola',
    'PIB_Pesca',
    'PIB_Mineria',
    'PIB_Mineria_del_cobre',
    'PIB_Otras_actividades_mineras',
    'PIB_Industria_Manufacturera',
    'PIB_Alimentos',
    'PIB_Bebidas_y_tabaco',
    'PIB_Textil',
    'PIB_Maderas_y_muebles',
    'PIB_Celulosa',
    'PIB_Refinacion_de_petroleo',
    'PIB_Quimica',
    'PIB_Minerales_no_metalicos_y_metalica_basica',
    'PIB_Productos_metalicos',
    'PIB_Electricidad',
    'PIB_Construccion',
    'PIB_Comercio',
    'PIB_Restaurantes_y_hoteles',
    'PIB_Transporte',
    'PIB_Comunicaciones',
    'PIB_Servicios_financieros',
    'PIB_Servicios_empresariales',
    'PIB_Servicios_de_vivienda',
    'PIB_Servicios_personales',
    'PIB_Administracion_publica',
    'PIB_a_costo_de_factores',
    'PIB',
    'Imacec_empalmado',
    'Imacec_produccion_de_bienes',
    'Imacec_minero',
    'Imacec_industria',
    'Imacec_resto_de_bienes',
    'Imacec_comercio',
    'Imacec_servicios',
    'Imacec_a_costo_de_factores',
    'Imacec_no_minero',
    'Indice_de_ventas_comercio_real_no_durables_IVCM',
]


In [ ]:
df_merge = pd.read_csv('merge_data.csv')

In [ ]:
    df_merge['Periodo'] = df_merge.apply(lambda x: str(x.anio) + '-' + str(x.mes), axis=1)
    df_merge.set_index('Periodo', inplace=True)

In [163]:
    pipe =  Pipeline([
        ('fixing_pib_vars', FixingFormattedString(PIB_COLS, 'PIB')),
        ('fixing_imacec_indice_vars', FixingFormattedString(IMACEC_INDICE_COLS, 'IMACEC_INDICE')),
        ('rolling_with_mean', RollingTransformer(CITY_COLS + PIB_COLS + IMACEC_INDICE_COLS, 'mean')),
        ('rolling_with_std', RollingTransformer(CITY_COLS + PIB_COLS + IMACEC_INDICE_COLS, 'std')),
        ('take_vars_before_scaler', TakeVariables(TAKE_VARS)),
        ('dropna', DropNaTransformer()),
        # ('scale', StandardScaler()),
        # ('selector', SelectKBest()),
        # ('polynomial_features', PolynomialFeatures()),
    ])


    df_prec = pipe.fit_transform(df_merge.drop('Precio_leche', axis=1), y=df_merge.Precio_leche)

In [166]:
df_prec[0]

,Coquimbo_rolling3_mean,Valparaiso_rolling3_mean,Metropolitana_de_Santiago_rolling3_mean,Libertador_Gral__Bernardo_O_Higgins_rolling3_mean,Maule_rolling3_mean,Biobio_rolling3_mean,La_Araucania_rolling3_mean,Los_Rios_rolling3_mean,PIB_Agropecuario_silvicola_rolling3_mean,PIB_Pesca_rolling3_mean,PIB_Mineria_rolling3_mean,PIB_Mineria_del_cobre_rolling3_mean,PIB_Otras_actividades_mineras_rolling3_mean,PIB_Industria_Manufacturera_rolling3_mean,PIB_Alimentos_rolling3_mean,PIB_Bebidas_y_tabaco_rolling3_mean,PIB_Textil_rolling3_mean,PIB_Maderas_y_muebles_rolling3_mean,PIB_Celulosa_rolling3_mean,PIB_Refinacion_de_petroleo_rolling3_mean,PIB_Quimica_rolling3_mean,PIB_Minerales_no_metalicos_y_metalica_basica_rolling3_mean,PIB_Productos_metalicos_rolling3_mean,PIB_Electricidad_rolling3_mean,PIB_Construccion_rolling3_mean,PIB_Comercio_rolling3_mean,PIB_Restaurantes_y_hoteles_rolling3_mean,PIB_Transporte_rolling3_mean,PIB_Comunicaciones_rolling3_mean,PIB_Servicios_financieros_rolling3_mean,PIB_Servicios_empresariales_rolling3_mean,PIB_Servicios_de_vivienda_rolling3_mean,PIB_Servicios_personales_rolling3_mean,PIB_Administracion_publica_rolling3_mean,PIB_a_costo_de_factores_rolling3_mean,PIB_rolling3_mean,Imacec_empalmado_rolling3_mean,Imacec_produccion_de_bienes_rolling3_mean,Imacec_minero_rolling3_mean,Imacec_industria_rolling3_mean,Imacec_resto_de_bienes_rolling3_mean,Imacec_comercio_rolling3_mean,Imacec_servicios_rolling3_mean,Imacec_a_costo_de_factores_rolling3_mean,Imacec_no_minero_rolling3_mean,Indice_de_ventas_comercio_real_no_durables_IVCM_rolling3_mean,Coquimbo_rolling3_std,Valparaiso_rolling3_std,Metropolitana_de_Santiago_rolling3_std,Libertador_Gral__Bernardo_O_Higgins_rolling3_std,Maule_rolling3_std,Biobio_rolling3_std,La_Araucania_rolling3_std,Los_Rios_rolling3_std,PIB_Agropecuario_silvicola_rolling3_std,PIB_Pesca_rolling3_std,PIB_Mineria_rolling3_std,PIB_Mineria_del_cobre_rolling3_std,PIB_Otras_actividades_mineras_rolling3_std,PIB_Industria_Manufacturera_rolling3_std,PIB_Alimentos_rolling3_std,PIB_Bebidas_y_tabaco_rolling3_std,PIB_Textil_rolling3_std,PIB_Maderas_y_muebles_rolling3_std,PIB_Celulosa_rolling3_std,PIB_Refinacion_de_petroleo_rolling3_std,PIB_Quimica_rolling3_std,PIB_Minerales_no_metalicos_y_metalica_basica_rolling3_std,PIB_Productos_metalicos_rolling3_std,PIB_Electricidad_rolling3_std,PIB_Construccion_rolling3_std,PIB_Comercio_rolling3_std,PIB_Restaurantes_y_hoteles_rolling3_std,PIB_Transporte_rolling3_std,PIB_Comunicaciones_rolling3_std,PIB_Servicios_financieros_rolling3_std,PIB_Servicios_empresariales_rolling3_std,PIB_Servicios_de_vivienda_rolling3_std,PIB_Servicios_personales_rolling3_std,PIB_Administracion_publica_rolling3_std,PIB_a_costo_de_factores_rolling3_std,PIB_rolling3_std,Imacec_empalmado_rolling3_std,Imacec_produccion_de_bienes_rolling3_std,Imacec_minero_rolling3_std,Imacec_industria_rolling3_std,Imacec_resto_de_bienes_rolling3_std,Imacec_comercio_rolling3_std,Imacec_servicios_rolling3_std,Imacec_a_costo_de_factores_rolling3_std,Imacec_no_minero_rolling3_std,Indice_de_ventas_comercio_real_no_durables_IVCM_rolling3_std,Coquimbo,Valparaiso,Metropolitana_de_Santiago,Libertador_Gral__Bernardo_O_Higgins,Maule,Biobio,La_Araucania,Los_Rios,PIB_Agropecuario_silvicola,PIB_Pesca,PIB_Mineria,PIB_Mineria_del_cobre,PIB_Otras_actividades_mineras,PIB_Industria_Manufacturera,PIB_Alimentos,PIB_Bebidas_y_tabaco,PIB_Textil,PIB_Maderas_y_muebles,PIB_Celulosa,PIB_Refinacion_de_petroleo,PIB_Quimica,PIB_Minerales_no_metalicos_y_metalica_basica,PIB_Productos_metalicos,PIB_Electricidad,PIB_Construccion,PIB_Comercio,PIB_Restaurantes_y_hoteles,PIB_Transporte,PIB_Comunicaciones,PIB_Servicios_financieros,PIB_Servicios_empresariales,PIB_Servicios_de_vivienda,PIB_Servicios_personales,PIB_Administracion_publica,PIB_a_costo_de_factores,PIB,Imacec_empalmado,Imacec_produccion_de_bienes,Imacec_minero,Imacec_industria,Imacec_resto_de_bienes,Imacec_comercio,Imacec_servicios,Imacec_a_costo_de_factores,Imacec_no_minero,Indice_de_ventas_comerci

In [157]:
        mask = (np.isnan(df_prec).any(1))
        if hasattr(df_prec, 'loc'):
            X = df_prec.loc[~mask]

In [158]:
X

,anio,mes,Coquimbo,Valparaiso,Metropolitana_de_Santiago,Libertador_Gral__Bernardo_O_Higgins,Maule,Biobio,La_Araucania,Los_Rios,PIB_Agropecuario_silvicola,PIB_Pesca,PIB_Mineria,PIB_Mineria_del_cobre,PIB_Otras_actividades_mineras,PIB_Industria_Manufacturera,PIB_Alimentos,PIB_Bebidas_y_tabaco,PIB_Textil,PIB_Maderas_y_muebles,PIB_Celulosa,PIB_Refinacion_de_petroleo,PIB_Quimica,PIB_Minerales_no_metalicos_y_metalica_basica,PIB_Productos_metalicos,PIB_Electricidad,PIB_Construccion,PIB_Comercio,PIB_Restaurantes_y_hoteles,PIB_Transporte,PIB_Comunicaciones,PIB_Servicios_financieros,PIB_Servicios_empresariales,PIB_Servicios_de_vivienda,PIB_Servicios_personales,PIB_Administracion_publica,PIB_a_costo_de_factores,PIB,Imacec_empalmado,Imacec_produccion_de_bienes,Imacec_minero,Imacec_industria,Imacec_resto_de_bienes,Imacec_comercio,Imacec_servicios,Imacec_a_costo_de_factores,Imacec_no_minero,Indice_de_ventas_comercio_real_no_durables_IVCM,Coquimbo_rolling3_mean,Valparaiso_rolling3_mean,Metropolitana_de_Santiago_rolling3_mean,Libertador_Gral__Bernardo_O_Higgins_rolling3_mean,Maule_rolling3_mean,Biobio_rolling3_mean,La_Araucania_rolling3_mean,Los_Rios_rolling3_mean,PIB_Agropecuario_silvicola_rolling3_mean,PIB_Pesca_rolling3_mean,PIB_Mineria_rolling3_mean,PIB_Mineria_del_cobre_rolling3_mean,PIB_Otras_actividades_mineras_rolling3_mean,PIB_Industria_Manufacturera_rolling3_mean,PIB_Alimentos_rolling3_mean,PIB_Bebidas_y_tabaco_rolling3_mean,PIB_Textil_rolling3_mean,PIB_Maderas_y_muebles_rolling3_mean,PIB_Celulosa_rolling3_mean,PIB_Refinacion_de_petroleo_rolling3_mean,PIB_Quimica_rolling3_mean,PIB_Minerales_no_metalicos_y_metalica_basica_rolling3_mean,PIB_Productos_metalicos_rolling3_mean,PIB_Electricidad_rolling3_mean,PIB_Construccion_rolling3_mean,PIB_Comercio_rolling3_mean,PIB_Restaurantes_y_hoteles_rolling3_mean,PIB_Transporte_rolling3_mean,PIB_Comunicaciones_rolling3_mean,PIB_Servicios_financieros_rolling3_mean,PIB_Servicios_empresariales_rolling3_mean,PIB_Servicios_de_vivienda_rolling3_mean,PIB_Servicios_personales_rolling3_mean,PIB_Administracion_publica_rolling3_mean,PIB_a_costo_de_factores_rolling3_mean,PIB_rolling3_mean,Imacec_empalmado_rolling3_mean,Imacec_produccion_de_bienes_rolling3_mean,Imacec_minero_rolling3_mean,Imacec_industria_rolling3_mean,Imacec_resto_de_bienes_rolling3_mean,Imacec_comercio_rolling3_mean,Imacec_servicios_rolling3_mean,Imacec_a_costo_de_factores_rolling3_mean,Imacec_no_minero_rolling3_mean,Indice_de_ventas_comercio_real_no_durables_IVCM_rolling3_mean,Coquimbo_rolling3_std,Valparaiso_rolling3_std,Metropolitana_de_Santiago_rolling3_std,Libertador_Gral__Bernardo_O_Higgins_rolling3_std,Maule_rolling3_std,Biobio_rolling3_std,La_Araucania_rolling3_std,Los_Rios_rolling3_std,PIB_Agropecuario_silvicola_rolling3_std,PIB_Pesca_rolling3_std,PIB_Mineria_rolling3_std,PIB_Mineria_del_cobre_rolling3_std,PIB_Otras_actividades_mineras_rolling3_std,PIB_Industria_Manufacturera_rolling3_std,PIB_Alimentos_rolling3_std,PIB_Bebidas_y_tabaco_rolling3_std,PIB_Textil_rolling3_std,PIB_Maderas_y_muebles_rolling3_std,PIB_Celulosa_rolling3_std,PIB_Refinacion_de_petroleo_rolling3_std,PIB_Quimica_rolling3_std,PIB_Minerales_no_metalicos_y_metalica_basica_rolling3_std,PIB_Productos_metalicos_rolling3_std,PIB_Electricidad_rolling3_std,PIB_Construccion_rolling3_std,PIB_Comercio_rolling3_std,PIB_Restaurantes_y_hoteles_rolling3_std,PIB_Transporte_rolling3_std,PIB_Comunicaciones_rolling3_std,PIB_Servicios_financieros_rolling3_std,PIB_Servicios_empresariales_rolling3_std,PIB_Servicios_de_vivienda_rolling3_std,PIB_Servicios_personales_rolling3_std,PIB_Administracion_publica_rolling3_std,PIB_a_costo_de_factores_rolling3_std,PIB_rolling3_std,Imacec_empalmado_rolling3_std,Imacec_produccion_de_bienes_rolling3_std,Imacec_minero_rolling3_std,Imacec_industria_rolling3_std,Imacec_resto_de_bienes_rolling3_std,Imacec_comercio_rolling3_std,Imacec_servicios_rolling3_std,Imacec_a_costo_de_factores_rolling3_std,Imacec_no_minero_rolling3_std,Indice_de_ventas_comercio_re

In [148]:
(np.isnan(df_prec).any(1))

Periodo
2014.0-1.0      True
2014.0-2.0     False
2014.0-3.0     False
2014.0-4.0     False
2014.0-5.0     False
               ...  
2019.0-11.0    False
2019.0-12.0    False
2020.0-1.0     False
2020.0-2.0     False
2020.0-3.0     False
Length: 75, dtype: bool

In [133]:
pd.DataFrame(df_prec)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137
0,-0.617655,-0.720988,-0.710609,-0.975117,-1.080248,-0.977595,-0.769239,-0.831501,0.029738,-0.265226,0.129060,-0.224392,-0.306838,0.127862,0.315241,0.203133,-2.180316,-1.974896,0.536608,0.664945,0.240536,-2.055392,-0.364957,-0.072961,-0.150875,-0.573504,-2.434640,-0.090680,-0.375550,0.248564,0.432370,-1.526765,2.457483,-0.087979,6.015491,-0.240961,-2.393765,-0.643297,-1.072139,-1.630400,0.005755,-0.661678,-2.795758,-2.407429,-2.380129,-1.851031,-0.686017,-0.756748,-0.701831,-1.039255,-1.177981,-1.205476,-0.598618,-0.605480,2.726623,-1.090001,-0.620592,-0.348215,-0.310721,-0.526481,-0.878027,-0.733023,2.028106,1.232238,1.150520,1.436417,-0.279825,1.562054,-0.426449,-0.409376,-1.065583,-0.092268,1.672368,-0.780624,-0.491172,-0.744562,-0.430312,1.354768,-0.540841,-0.561818,5.317651,-0.618395,-0.639606,-1.133749,-0.912778,0.395442,-1.811451,-1.275384,-0.485291,-0.667911,-0.718280,-0.771079,-0.463622,-0.446228,-0.413150,-0.729329,-0.887785,-0.850553,-0.867546,-0.926890,1.388660,0.035847,0.039118,-0.174833,-0.216055,-0.022982,0.184440,0.015856,-2.879882,-2.244548,1.333512,1.553902,-0.150085,0.135544,-0.391215,-0.223855,-0.100841,-0.231113,-2.755871,-0.079278,-0.372120,0.120130,0.070139,0.196785,1.590770,-0.035733,7.963176,-0.222753,-2.246703,-0.763401,-1.026589,-1.838642,-0.014358,-0.575546,-2.705120,-2.265111,-2.255547,-1.177226
1,-0.625043,-0.724879,-0.778545,-0.916735,-0.954187,-0.783409,-0.557933,-0.603892,-0.600989,-1.243075,0.241729,-0.202670,-0.274519,0.354539,0.708870,0.216249,-1.360044,-1.151640,-0.086614,0.096295,0.432607,-0.994118,-0.229730,-0.083980,0.059187,-0.449776,-1.639651,-0.007620,-0.388416,0.225196,0.354553,-0.902121,1.428178,-0.077713,4.025379,-0.023200,-1.685992,-0.203002,-0.680186,-0.728590,0.194783,-0.108936,-2.189491,-1.674991,-1.677241,-1.413223,-0.707039,-0.816501,-0.729194,-0.938184,-0.879673,-0.706080,-0.478050,-0.444002,1.977952,0.102373,-0.568178,-0.332643,-0.329000,-0.437013,-0.383671,-0.781767,1.587324,0.900952,0.640332,0.932870,-0.368758,1.309208,-0.492934,-0.503829,-0.856781,-0.138515,1.177448,-0.710396,-0.569386,-0.720333,-0.496393,0.934593,1.058584,-0.554508,4.228473,-0.459156,0.899783,-0.077013,-0.525844,1.426951,-0.862484,0.235188,0.862010,1.035827,1.017183,-0.296582,-0.440872,-0.540743,-0.675845,-0.613454,-0.558364,-0.320455,-0.104378,-0.117438,-1.418987,-1.673649,0.304280,-0.115457,-0.107309,0.459878,0.840658,0.129435,0.188936,0.264763,-0.757229,-0.663980,0.427117,0.632827,0.013038,-0.076047,0.242002,-0.160570,-0.031192,0.088217,-0.241733,0.083945,0.117753,0.183884,-0.430042,-0.021731,0.045812,0.233028,-0.244993,0.512791,0.064477,0.752799,0.469904,0.744505,-0.849924,-0.199479,-0.250732,-0.330047
2,-0.676414,-0.729178,-0.737291,-0.833833,-0.751781,-0.563485,-0.516278,-0.551518,0.032793,-0.001599,0.235345,-0.204185,-0.271328,0.475680,0.971057,0.310317,-1.124807,-0.993735,1.133479,0.088075,0.481806,0.783820,-0.071315,-0.225451,0.334802,-0.255026,-1.657416,0.020458,-0.438456,0.173652,0.302601,0.361051,0.324192,-0.073743,3.602854,0.093546,-1.307074,-0.252134,-0.702226,-0.242168,0.013579,0.015446,-1.607337,-1.258035,-1.272935,-1.304221,-0.724706,-0.808652,-0.746657,-0.937802,-0.670103,-0.548112,-0.443215,-0.401664,1.532459,1.453360,-0.567972,-0.332337,-0.327335,-0.404387,-0.457727,-0.683426,1.788931,1.001839,0.650805,0.940802,-0.339879,-0.693250,-0.386645,-0.633334,-0.772027,-0.468129,1.166614,-0.713715,-0.622718,-0.697875,-0.550211,-0.897457,0.869726,-0.556931,4.496957,-0.450423,0.987377,-0.031468,-0.523876,1.795984,-0.200768,0.106494,1.470463,1.146381,1.122571,-

In [120]:
valid_cols = [col for col in TAKE_VARS if col in df_merge.columns]

In [97]:
d1 = pd.DataFrame(df_merge['Coquimbo'].rolling(window=3, min_periods=1).mean())

In [99]:
d1

,Coquimbo
Periodo,
2014.0-1.0,3.524673
2014.0-2.0,2.606765
2014.0-3.0,2.483577
2014.0-4.0,1.626952
2014.0-5.0,1.892314
...,...
2019.0-11.0,3.081688
2019.0-12.0,0.723954
2020.0-1.0,1.199760


In [66]:
d1 = df_merge[['Coquimbo']].rolling(window=3, min_periods=1).mean()#.reset_index()
d2 = df_merge['Valparaiso'].rolling(window=3, min_periods=1).mean().reset_index()

In [67]:
d1

,Coquimbo
0,3.524673
1,2.606765
2,2.483577
3,1.626952
4,1.892314
...,...
70,3.081688
71,0.723954
72,1.199760
73,1.255636
